In [4]:
import argparse
from collections import Counter
from typing import List
from openai import OpenAI
from pydantic import BaseModel
from sklearn.cluster import DBSCAN
import numpy as np
import json
from prisma import Json, Prisma
from prisma.models import DeduplicatedCard, ValuesCard
from prisma.enums import ProcessState
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_distances

In [8]:
import sys

sys.path.append("../modules")
from llms import gpt4

In [11]:
from deduplicate import _cluster, ClusterableObject, client

In [12]:
db = Prisma()

In [14]:
if not db.is_connected():
    db.connect()

# Find all edges that are not deduplicated yet.
edges = db.edge.find_many(
    where={
        "generationId": 45,
    
    }
)
# Deduplicate contexts.
contexts = list([e.contextName for e in edges])

print(len(contexts))

300


In [16]:
# 
# embeddings = client.embeddings.create(
#     model="text-embedding-3-large", input=[c.lower() for c in contexts]
# )

In [72]:
cluster_objs = [
    ClusterableObject(id=i, embedding=embeddings.data[i].embedding)
    for i in range(len(contexts))
]
clusters = [
    [contexts[c.id] for c in cluster]
    for cluster in _cluster(cluster_objs, eps=0.22, min_samples=1)
]

print(len(clusters))

105


In [73]:
len(clusters)

105

In [74]:
# print all clusters where the length is longer than 1
for c in clusters:
    if len(c) > 1:
        print(c)

['boundary-pushing jokes', 'boundary-pushing jokes', 'boundary-pushing comedic material', 'boundary-pushing comedic material']
['mischievous schemes', 'mischievous schemes', 'mischievous pranks', 'mischievous pranks', 'cunning schemes', 'cunning schemes', 'mischievous pranks', 'mischievous pranks', 'mischievous adventures', 'mischievous adventures', 'mischievous pranks', 'mischievous pranks', 'mischievous pranks', 'mischievous pranks', 'mischievous pranks', 'mischievous pranks', 'mischievous adventures', 'mischievous adventures', 'mischievous pranks', 'mischievous pranks', 'clever schemes', 'clever schemes', 'mischievous pranks', 'mischievous pranks', 'mischievous activities', 'mischievous activities', 'mischievous pranks', 'mischievous pranks', 'clever schemes', 'clever schemes', 'mischievous pranks', 'mischievous pranks', 'mischievous schemes', 'mischievous schemes', 'mischievous pranks', 'mischievous pranks']
['pop culture fascinations', 'pop culture fascinations']
['societal norm c